In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, isnan, when, count
import pandas as pd
from pyspark.sql.functions import split, unix_timestamp
from datetime import time

In [17]:
spark=SparkSession.builder.appName("first big data").getOrCreate()

In [27]:
df_sp=spark.read.csv("/content/sample_data/yellow_tripdata_2015-01.csv",header=True, inferSchema=True)

# Yeni Bölüm

In [ ]:
df_sp.show(5)

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       2| 2015-01-15 19:05:39|  2015-01-15 19:23:42|              1|         1.59|  -73.993896484375|  40.7501106262207|         1|    

In [ ]:
df_sp.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- pickup_longitude: double (nullable = true)
 |-- pickup_latitude: double (nullable = true)
 |-- RateCodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- dropoff_longitude: double (nullable = true)
 |-- dropoff_latitude: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)



In [ ]:
byk=df_sp.select([count(when(col(c).isNull(),c)).alias(c)for c in df_sp.columns]).show()

+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|pickup_longitude|pickup_latitude|RateCodeID|store_and_fwd_flag|dropoff_longitude|dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------------+---------------+----------+------------------+-----------------+----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       0|                   0|                    0|              0|            0|               0|              0|         0|                 0|              

In [ ]:
df_sp.summary().show()

+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+-------------------+--------------------+------------------+-------------------+---------------------+------------------+
|summary|          VendorID|   passenger_count|     trip_distance|  pickup_longitude|  pickup_latitude|        RateCodeID|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|       payment_type|       fare_amount|              extra|             mta_tax|        tip_amount|       tolls_amount|improvement_surcharge|      total_amount|
+-------+------------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+-------------------+------------------+-------------------+--------------------+------------------+-------------------+---------------------+---

In [ ]:
select=df_sp.select('fare_amount','total_amount')
select.show(5)

+-----------+------------+
|fare_amount|total_amount|
+-----------+------------+
|       12.0|       17.05|
|       14.5|        17.8|
|        9.5|        10.8|
|        3.5|         4.8|
|       15.0|        16.3|
+-----------+------------+
only showing top 5 rows



In [ ]:
flr=df_sp.filter(df_sp['total_amount']<2)
flr.show(5)

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude| dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+-----------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2015-01-28 20:22:19|  2015-01-28 20:23:19|              2|          4.8|-74.03569030761719|40.743648529052734|         5|       

In [ ]:
bahsıs=df_sp.orderBy(df_sp['tip_amount'], ascending=False)
bahsıs.show(5)

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2015-01-25 07:22:54|  2015-01-25 07:26:12|              1|          0.8|-73.98861694335938|40.778526306152344|         1|    

In [ ]:
sıra=df_sp.orderBy(df_sp['tip_amount'], ascending=True)
sıra.show(5)

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|      date|    time|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------+
|       2| 2015-01-14 11:52:09|  2015-01-14 11:52:20|              1|       

In [ ]:
passenger=df_sp.orderBy(df_sp['passenger_count'], ascending=False)
passenger.show(5)

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|      date|    time|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+----------+--------+
|       2| 2015-01-02 20:14:40|  2015-01-02 20:14:44|              9|       

In [26]:
df_sp = df_sp.withColumn("time", split(df_sp["tpep_pickup_datetime"], " ")[1])\
             .withColumn("timee", split(df_sp["tpep_dropoff_datetime"], " ")[1])

df_sp = df_sp.withColumn("time_different_minute",
                         (unix_timestamp("tpep_dropoff_datetime", "yyyy-MM-dd HH:mm:ss") -
                          unix_timestamp("tpep_pickup_datetime", "yyyy-MM-dd HH:mm:ss")) / 60)


df_sp.select("tpep_pickup_datetime", "tpep_dropoff_datetime", "time_different_minute")
df_sp.show(5)


+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+--------+---------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|    time|   timee|time_different_minute|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------+--------+---------------------+
|       2| 2015-

In [6]:
tip=df_sp.select('tip_amount', 'passenger_count')
tip.show(5)

+----------+---------------+
|tip_amount|passenger_count|
+----------+---------------+
|      3.25|              1|
|       2.0|              1|
|       0.0|              1|
|       0.0|              1|
|       0.0|              1|
+----------+---------------+
only showing top 5 rows



In [ ]:
odemey=df_sp.olderBy(df_sp['payment_type'], ascending=False)
odemey.show(5)

AttributeError: 'DataFrame' object has no attribute 'olderBy'

In [ ]:
gecıs=df_sp.orderBy(df_sp['tolls_amount'], ascending=False)
gecıs.show(5)

+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|  pickup_longitude|   pickup_latitude|RateCodeID|store_and_fwd_flag| dropoff_longitude|  dropoff_latitude|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+------------------+------------------+----------+------------------+------------------+------------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2015-01-01 07:02:06|  2015-01-01 07:33:58|              2|         19.7| -73.9880142211914|40.723934173583984|         3|    

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful